In [61]:
import os, glob, string, re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import pandas as pd
from collections import Counter

# Extracting the Data
Know that we have learned that we will not be able to process the data using a Spark pipeline, we will implement here a Python pipeline to read the files.

## 1 - Extracting the paths

We first have to get all the paths containing the XML files.

In [18]:
GDL = 'GDL'
JDG = 'JDG'
listFiles = []

#Go through all the files of both journals
for root, dirs, files in os.walk(GDL): 
    for file in files:
        if file.endswith('.xml'):
             listFiles.append(os.path.join(root, file))

for root, dirs, files in os.walk(JDG): 
    for file in files:
        if file.endswith('.xml'):
             listFiles.append(os.path.join(root, file))

Now that we have the path for each file in the dataset, we can work through each of them and count words. We will use the method described in **Filtering the Data.ipynb**

## 2 - Extracting the word count
Know that we have the paths for all text files, we can read the text and then clean it.

First we need the stop words.

In [28]:
stop_words = list(stopwords.words('french'))
stop_words.extend(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']',
                   '{', '}', '<', '>', '...', '-', '•', '/', '%', '«', '»', 'le',
                  'les','..','©','plus','*','``','av','p.','fr','cette','a.','v.','b.','d.',
                   'c.','e.','f.','i.','g.','h.','i.','j.','l.','m.','n.','o.','q.','r.','q.',
                   's.','t.','u.','v.','w.','x.','z.','y.',"d'un","d'une",'dm',"c'est",
                   "jusqu'au",'entre','comme','si','di','„','&',"qu'il",'_','fd',"n'a",'alors',
                   "s'est","n'est",'cs','dès','où',"jusqu'à",'déjà',"''",'|','£','®','+','-«',
                   '--','.-'])

stop_words.extend(['','-.','.,','(,','»,',').','-.','av.','».',
                      '....','..','...','.....','()','(r).','—',')-«',
                      '".','-,','(-.',')-.','(©'])

And preparing the dictionaries.

In [31]:
listPunct = list(string.punctuation)
listPunct.remove("'")
listPunct.remove("-")
#we do not want to stick together all the words that
#have an apostrophe (l'armée should not become larmée)
stopDict = dict(zip(listPunct,['']*len(listPunct)))

We also need a constant which tells us how many of the most frequenc words we take.

In [32]:
numMostFreq = 2000

And then we can go through the list to extract the data in each file. We also set a counter to see how long it takes.

In [56]:
total = len(listFiles)
allText = []

In [62]:
for idx, file in enumerate(listFiles):
    #counter 
    counter(idx)
    #find the date at which the article is published.
    date = pd.to_datetime(file[4:11])
    #Open the XML
    f = open(file,'r')
    soupArticle = BeautifulSoup(f,'lxml')
    f.close()
    
    #Extracting the entire text from the XML
    Article = []
    for ft in soupArticle.find_all('full_text'):
        #Here we lower each words and we remove the numbers from the text
        Article.append(re.sub(r'\d+', '', ft.text).lower())

    textArticle = ' '.join(Article)  
    
    #Cleaning and counting
    for k,v in stopDict.items():
        textArticle = textArticle.replace(k,v)

    dictCounter = [wordsDict for wordsDict in textArticle.split(' ') if wordsDict not in stop_words]
    countDict = Counter(dictCounter).most_common(numMostFreq)
    
    #Transofming to a dataframe
    df = pd.DataFrame(countDict).transpose()
    df.columns = df.loc[0]
    df.drop(0,inplace=True)
    df.index = [date]
    
    allText.append(df)

We are at : 0.0  %
We are at : 2.306805074971165  %
We are at : 4.61361014994233  %
We are at : 6.920415224913495  %
We are at : 9.22722029988466  %
We are at : 11.534025374855824  %
We are at : 13.84083044982699  %
We are at : 16.147635524798154  %
We are at : 18.45444059976932  %
We are at : 20.761245674740483  %
We are at : 23.06805074971165  %
We are at : 25.37485582468281  %
We are at : 27.68166089965398  %
We are at : 29.988465974625143  %
We are at : 32.29527104959631  %
We are at : 34.602076124567475  %
We are at : 36.90888119953864  %
We are at : 39.21568627450981  %
We are at : 41.522491349480966  %
We are at : 43.82929642445214  %
We are at : 46.1361014994233  %
We are at : 48.44290657439446  %
We are at : 50.74971164936562  %
We are at : 53.05651672433679  %
We are at : 55.36332179930796  %
We are at : 57.67012687427913  %
We are at : 59.976931949250286  %
We are at : 62.28373702422145  %
We are at : 64.59054209919262  %
We are at : 66.89734717416378  %
We are at : 69.20415

In [63]:
test = allText[0:5]

In [ ]:
dfAllText = pd.concat(allText,axis=0)

In [ ]:
dfAllText.head()

In [ ]:
1+1

# Fucntions for the extractions

In [54]:
def counter(i):
    if i%100==0:
        print('We are at :', i/total*100 ,' %')